# Tutorial: Quantum Embedding Theory (QDET) Calculations with WEST

This tutorial shows how to compute correlated excitations of the $\mathrm{NV^-}$ defect in Diamond using the Quantum Defect Embedding Theory (QDET). In QDET, a set of selected KS orbitals forms the active space. The effective Hamiltonian that describes excitations within the active space is formulated as:
$$
\mathrm{H}^{eff} = \sum_{ij}^A t^{eff}_{ij}\hat{a}_i^\dagger \hat{a}_j + \frac{1}{2} \sum_{ijkl}^A v^{eff}_{ijkl} \hat{a}_i^\dagger \hat{a}_j^\dagger \hat{a}_l \hat{a}_k,
$$
where $t^{eff}$ and $v^{eff}$ are the effective one- and two-body terms, respectively. Exact diagonalization of the effective Hamiltonian yields the excitations of the spin defect. More details about the method, and definitions for $t^{eff}$ and $v^{eff}$ can be found in <a href="https://doi.org/10.1021/acs.jctc.2c00240">N. Sheng, C. Vorwerk, M. Govoni, and G. Galli, J. Chem. Theory Comput. 18, 6, 3512 (2022)</a>.

QDET calculations are performed in two separate steps:
1. The parameters $t^{eff}$ and $v^{eff}$ are calculated from first principles and written to file.
2. The effective Hamiltonian is constructed from the parameters read from file. Exact diagonalization is used to find the lowest eigenvalues and eigenvectors of the effective Hamiltonian.

## Step 1: Parameters of the Effective Hamiltonian

The parameters of the effective Hamiltonian $\mathrm{H}^{eff}$ are calculated from first principles by:
- 1.1 Carrying out a mean-field starting point calculation using DFT
- 1.2 Computing the projective dielectric eigendcomposition (PDEP)
- 1.3 Computing $t^{eff}$ and $v^{eff}$ for a chosen active space, and saving it them to file

### Step 1.1:  Mean-field Starting Point

As a first step, we have to perform the mean-field electronic structure calculations within density-functional theory (DFT) using the <a href='https://www.quantum-espresso.org/'>Quantum ESPRESSO</a> code.

Download the following files in your working directory:

In [1]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.0.upf

We can now inspect the `pw.in` file, the input for the `pw.x` code:

In [2]:
%%bash
cat pw.in

&CONTROL
calculation = 'scf'
wf_collect = .true.
pseudo_dir = './'
/
&SYSTEM
input_dft = 'PBE'
ibrav = 0
ecutwfc = 50
nosym = .true.
tot_charge = -1
nspin = 1
nbnd = 176
occupations = 'from_input'
nat = 63
ntyp = 2
/
&ELECTRONS
conv_thr = 1D-08
/
K_POINTS gamma
CELL_PARAMETERS angstrom
7.136012  0.000000  0.000000
0.000000  7.136012  0.000000
0.000000  0.000000  7.136012
ATOMIC_SPECIES
C  12.01099968  C_ONCV_PBE-1.0.upf
N  14.00699997  N_ONCV_PBE-1.0.upf
ATOMIC_POSITIONS crystal
C    0.99996000  0.99996000  0.99996000
C    0.12495000  0.12495000  0.12495000
C    0.99905000  0.25039000  0.25039000
C    0.12350000  0.37499000  0.37499000
C    0.25039000  0.99905000  0.25039000
C    0.37499000  0.12350000  0.37499000
C    0.25039000  0.25039000  0.99905000
C    0.37499000  0.37499000  0.12350000
C    0.00146000  0.00146000  0.50100000
C    0.12510000  0.12510000  0.62503000
C    0.00102000  0.24944000  0.74960000
C    0.12614000  0.37542000  0.87402000
C    0.24944000  0.00102000  0.74960

We can now run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

We note that the calculation is spin unpolarized (i.e., `nspin=1`) in order to reduce spin contamination. The parameters `ecutwfc` and the size of the supercell may be increased to converge the results.

### Step 1.2: Calculation of Dielectric Screening

As for GW calculations with WEST, we first have to determine the dielectric screening before we can proceed to calculate the excitations of the spin defect. In WEST, the dielectric screening is obtained from the projective dielectric eigendecomposition (PDEP). The calculation with `wstat.x` requires an input file `wstat.in`.

You can download the input file with the following command:

In [4]:
%%bash 
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wstat.in

Once again, we can have a look at the input file:

In [5]:
%%bash
cat wstat.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001


As we can see, there are no input parameters in `wstat.in` specific to QDET. We can now execute `wstat.x` with the following command.

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat.in > wstat.out

The number of PDEPs, i.e, `n_pdep_eigen`, may be increased to converge the results.

### Step 1.3: QDET Calculation

The calculation of the paramters for the effective Hamiltonian is carried out by using the `wfreq.x` program. The program will: (i) compute the $\mathrm{G_0W_0}$ quasiparticle energies, (ii) compute the partially screened Coulomb potential, and (iii) finally compute the parameters of the effective Hamiltonian.

You can download the input file with the following command:

In [6]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.in

Let us again inspect the file `wfreq.in`:

In [7]:
%%bash
cat wfreq.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001

wfreq_control:
  wfreq_calculation: XWGQH
  macropol_calculation: C
  l_enable_off_diagonal: true
  n_pdep_eigen_to_use: 512
  qp_bands: [87, 122, 123, 126, 127, 128]
  n_refreq: 300
  ecut_refreq: 2.0


The QDET calculation is triggered by the keyword `H` (for effective Hamiltonian) in the variable `wfreq_calculation`. The KS states that define the active space are selected with the variable `qp_bands` (in our case states 87, 122, 123, 126, 127, and 128). The active space may be chosen to be a set of localized defect orbitals in the supercell. For the $\mathrm{NV}^-$ center in the 63-atom supercell, we have chosen the orbitals with the largest localization factor. See the tutorial on the calculation of the localization factor <a href="http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/basic_006.ipynb">here</a>. 

For QDET, it is mandatory to set the flag `l_enable_off_diagonal : true` which allows for the calculation of off-diagonal entries of the GW self-energy matrix.

The `wfreq.x` calculation can be performed as

In [ ]:
%%bash
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out

If the reader does not have the computational ressource to run the calculation, the WEST JSON file can be downloaded as

In [8]:
%%bash
mkdir west.wfreq.save
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.json -O west.wfreq.save/wfreq.json

## Step 2: Exact Diagonalization of the Effective Hamiltonian

The effective Hamiltonian is constructed and diagonalized using the WESTpy code.

In [9]:
import json
import numpy as np

from westpy.qdet import QDETResult

# construct object for effective Hamiltonian
effective_hamiltonian = QDETResult(filename='west.wfreq.save/wfreq.json')

# diagonalize Hamiltonian
solution = effective_hamiltonian.solve()

 
 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  4.3.0
Today            :  2022-08-30 17:54:48.543733
-----------------------------------------------------
Building effective Hamiltonian...
nspin: 1
occupations: [[2. 2. 2. 2. 1. 1.]]


diag[1RDM - 1RDM(GS)]                                      
  E [eV] char                    87     122     123     126    127    128
0  0.000   3-                 0.000   0.000   0.000   0.000  0.000  0.000
1  0.436   1-                -0.001  -0.009  -0.018  -0.067  0.037  0.058
2  0.436   1-                -0.001  -0.009  -0.018  -0.067  0.058  0.037
3  1.250   1-                -0.002  -0.019  -0.023  -0.067  0.056  0.055
4  1.941   3-                -0.003  -0.010  -0.127  -0.860  0.000  1.000
5  1.941   3-                -0.003  -0.010  -0.127  -0.860  1.000  0.000
6  2.937   1-                -0.000  -0.032  -0.043  -0.855  0.110  0.821
7  2.937   1-                -0.000  -0.033  -0.043  -0.855  0.821  0.110
8  4.662   1-                -0.005  -0.055  -0.188  -1.672  0.960  0.960
9  5.073   3-                -0.014  -0.702  -0.209  -0.075  0.000  1.000

-----------------------------------------------------


Calling the function `solve()` of the object `QDETResult` writes the excitation energies (in eV) to screen, spin multiplicity and relative occupation (compared to the occupation of the groundstate) to screen.

Let us consider the first excited state (`# 1`) as an example: It has an energy of `0.436 eV` above the groundstate, has spin multiplicity `1` (i.e., it is a singlet), and is two-fold degenerate because the state `# 2` has the same energy. It has nearly the same occupation as the groundstate, only the occupation in band 126 is reduced by 0.1, while that of 128 is increased by 0.1. Since the state has nearly the same occupation as the groundstate, it is a *multiplet* excitation.

A more detailed information about the excitations is stored in the dictionary `solution`:

In [11]:
print(solution.keys())

dict_keys(['nstates', 'evs_au', 'evs', 'evcs', 'mults', 'rdm1s'])


Explanation of the keys:
- `nstates` : the number of excitations that have been calculated.
- `evs` : excitation energies relative to the groundstate (in eV).
- `evs_au`: absolute energies of the many-body states (in Hartree).
- `mults`: spin multiplicity of each state.
- `rdm1s`: spin-summed reduced 1-body density matrix for each many-body state.
- `evcs`: many-body state in 2nd quantization (eigenvector of the effective Hamiltonian).